 # 抓取網址

In [ ]:
!pip install requests
# 安裝request

In [ ]:
url_template = "https://www.518.com.tw/job-index-P-%s.html?ab=2032001,2032002"

for page in range(1,40):
    url = url_template%page
    print(url)
    
# 抓取所有分頁網址 使用%S。回圈for跑出1-40頁分頁

In [ ]:
import requests

def get_page(url):
    resp = requests.get(url)
    return resp.text
#建立get_page方法

In [ ]:
def save2file(filename, text):
    with open(filename, 'w') as f:
        f.write(text)
#建立save方法

In [ ]:
!mkdir -p /home/ubuntu/518_data/list_page
#新建資料夾為儲存所用

In [ ]:
url_template = "https://www.518.com.tw/job-index-P-%s.html?ab=2032001,2032002"
for page in range(1,40):
    url = url_template%page
    print("[INFO]crawling %s"%url)
    text = get_page(url)
    save2file('/home/ubuntu/518_data/list_page/' + "%s.html"%page, text)
    
#透過上序方法將跑出的所有分頁儲存在資料夾中

In [ ]:
import re
import glob

glob.glob('/home/ubuntu/518_data/list_page/*')

res = []
for filename in glob.glob('/home/ubuntu/518_data/list_page/*'):
    with open(filename) as f:
        html = f.read()
        detail_urls = re.findall('''(https://www.518.com.tw/.+html).kw=&pi=..''', html)
        res.extend(detail_urls)

res

#抓取位於/home/ubuntu/518_data/list_page/裡所有資料

In [ ]:
!mkdir -p /home/ubuntu/518_data/detail_page
#新增一資料夾為儲存內工作內容所用

In [ ]:
for link in res:
    filename =link.split('job-')[1]
    url = 'https://www.518.com.tw/job-'+ filename
    print(url)
    
    # using the fucntions defined above ^^^^^^^^^^^^^
    text = get_page(url)
    save2file('/home/ubuntu/518_data/detail_page/' + filename, text)
    #透過分詞重新整理網址內容為最簡短化

In [ ]:
import glob

detail_page = glob.glob('/home/ubuntu/518_data/detail_page/*')[0]

print(detail_page)

with open(detail_page) as f:
    html = f.read()
    
html
#check detail_page裡的資料

In [ ]:
!pip install beautifulsoup4
# 安裝beautifulsoup4

In [ ]:
!pip install lxml
# install lxml

In [ ]:
from bs4 import BeautifulSoup


soup = BeautifulSoup(html, 'lxml') # html5lib
soup
#使用beautifulsoup察看資料

# 找出技術名稱

In [ ]:
skill_labels = [ele for ele in soup.select('.job-detail-box > dl > dt') if '擅長工具：' in ele.text]
#找到 擅長工具那行
skill_tools = skill_labels[0].next_sibling.next_sibling.text
#透過找到擅長工具那行找到鎖鑰資料
skill_tools

In [ ]:
res_dict = {}
for tool in glob.glob('/home/ubuntu/518_data/detail_page/*'):
    with open(tool) as t:
        html = t.read()
    skill_labels = [ele for ele in BeautifulSoup(html, 'lxml').select('.job-detail-box > dl > dt') if '擅長工具：' in ele.text]
    skill_tools = skill_labels[0].next_sibling.next_sibling.text
    name = tool.split('page/')[1].split('.')[0]
    res_dict[name] = skill_tools
res_dict

#透過回圈找出火有detail_page裡的擅長工具 并儲存於字典中

In [ ]:
new_res_dict='!'.join([ str(x) for x in res_dict.values()]).replace('、','!').replace(' ','').split('!')
new_res_dict

#將res_dit的value進行整理

In [ ]:
def cnt_skills(text, res={}):
    for word in new_res_dict:
        if word in res:
            res[word] += 1
        else:
            res[word] = 1
    return res
#製作計數方法

In [ ]:
res = cnt_skills(new_res_dict)
skills = [[word, res[word]]for word in res]

sorted_skills=sorted(skills, key=lambda x: x[1], reverse=True)
sorted_skills

#使用 cnt_skills方法並進行排序

# 找出職業名稱

In [ ]:
work_labes = [wn for wn in soup.select('.jobItem > dl > dt')if '職務類別：' in wn.text]
# 找出職務類別那行
work_name=work_labes[0].next_sibling.next_sibling.text
# 找出職務內容
work_name


In [ ]:
res_name = {}
for name in glob.glob('/home/ubuntu/518_data/detail_page/**'):
    with open(name) as n:
        html = n.read()
    work_labels = [wn for wn in BeautifulSoup(html, 'lxml').select('.jobItem > dl > dt')if '職務類別：' in wn.text]
    # 找出網頁內文 jobitem > dl > dt裡面的職務類別所有資料
    work_name=work_labels[0].next_sibling.next_sibling.text
    idname = name.split('page/')[1].split('.')[0]
    #將網頁整理成為id顯示用於儲存key使用
    res_name[idname] = work_name
        
res_name

In [ ]:
new_res_name='!'.join([ str(x) for x in res_name.values()]).replace('、','!').split('!')
new_res_name
#篩選value裏面資料,將兩筆以上資料做分開

In [ ]:
def cnt_name(text, res_name={}):
    for word in new_res_name:
        if word in res_name:
            res_name[word] += 1
        else:
            res_name[word] = 1
    return res_name
#定義計數方法

In [ ]:
res_name = cnt_name(new_res_name)
testname = [[word, res_name[word]]for word in res_name]


sort_name=sorted(testname, key=lambda x: x[1], reverse=True)
sort_name
#套用上序方法做計數,並使用sorted做排序

# 找出需要swift的職業名稱

In [ ]:
!mkdir -p /home/ubuntu/518_data/Swift_workename

In [ ]:
res_dict = {}
for tool in glob.glob('/home/ubuntu/518_data/detail_page/*'):
    with open(tool) as t:
        html = t.read()
    skill_labels = [ele for ele in BeautifulSoup(html, 'lxml').select('.job-detail-box > dl > dt') if '擅長工具：' in ele.text]
    skill_tools = skill_labels[0].next_sibling.next_sibling.text
    # 找出網頁內文 jobitem > dl > dt裡面的擅長工具所有資料
    name = tool.split('page/')[1].split('.')[0]
    #清理網頁使得name為該網頁id
    aa=skill_tools.split('、')
    if 'Swift' in aa :
        new_html = 'https://www.518.com.tw/job-' + name + '.html'
        print(new_html)
        text = get_page(new_html)
        save2file('/home/ubuntu/518_data/Swift_workename/' + name, text)
    #找尋tools裡含有swift的內容 並做出儲存及命名動作

In [ ]:
swift_res_name = {}
for name in glob.glob('/home/ubuntu/518_data/Swift_workename/*'):
    with open(name) as n:
        html = n.read()
        #讀取資料夾為存放網頁為上序條件的網頁
    swift_work_labels = [wn for wn in BeautifulSoup(html, 'lxml').select('.jobItem > dl > dt')if '職務類別：' in wn.text]
    swift_work_name=swift_work_labels[0].next_sibling.next_sibling.text
    idname = name.split('Swift_workename/')[1]
    swift_res_name[idname] = swift_work_name
    #找出上序網頁的職務類別
swift_res_name

In [ ]:
new_swift_res_name='!'.join([ str(x) for x in swift_res_name.values()]).replace('、','!').split('!')
new_swift_res_name
#針對找出的職務類別有兩筆以上的資料做出分割

In [ ]:
def cnt_swift_name(text, res_name={}):
    for word in new_swift_res_name:
        if word in res_name:
            res_name[word] += 1
        else:
            res_name[word] = 1
    return res_name
#定義計數方法

In [ ]:
res_name = cnt_swift_name(new_swift_res_name)
testname = [[word, res_name[word]]for word in res_name]


sort_swift_name=sorted(testname, key=lambda x: x[1], reverse=True)
sort_swift_name
#套用上序方法做計數,並使用sorted做排序

# 找出需要android的職務名稱

In [ ]:
!mkdir -p /home/ubuntu/518_data/android_workename

In [ ]:
res_dict = {}
for tool in glob.glob('/home/ubuntu/518_data/detail_page/*'):
    with open(tool) as t:
        html = t.read()
    skill_labels = [ele for ele in BeautifulSoup(html, 'lxml').select('.job-detail-box > dl > dt') if '擅長工具：' in ele.text]
    skill_tools = skill_labels[0].next_sibling.next_sibling.text
# 找出網頁內文 jobitem > dl > dt裡面的擅長工具所有資料
    name = tool.split('page/')[1].split('.')[0]
    aa=skill_tools.split('、')
    if 'Android' in aa :
        new_html = 'https://www.518.com.tw/job-' + name + '.html'
        print(new_html)
        text = get_page(new_html)
        save2file('/home/ubuntu/518_data/android_workename/' + name, text)
         #找尋tools裡含有android的內容 並做出儲存及命名動作

In [ ]:
android_res_name = {}
for name in glob.glob('/home/ubuntu/518_data/android_workename/*'):
    with open(name) as n:
        html = n.read()
        #讀取資料夾為存放網頁為上序條件的網頁
    android_work_labels = [wn for wn in BeautifulSoup(html, 'lxml').select('.jobItem > dl > dt')if '職務類別：' in wn.text]
    android_work_name = android_work_labels[0].next_sibling.next_sibling.text
    idname = name.split('android_workename/')[1]
    android_res_name[idname] = android_work_name
        #找出上序網頁的職務類別
android_res_name

In [ ]:
new_android_res_name='!'.join([ str(x) for x in android_res_name.values()]).replace('、','!').split('!')
new_android_res_name
#針對找出的職務類別有兩筆以上的資料做出分割

In [ ]:
def cnt_android_name(text, res_name={}):
    for word in new_android_res_name:
        if word in res_name:
            res_name[word] += 1
        else:
            res_name[word] = 1
    return res_name
#定義計數方法

In [ ]:
res_name = cnt_android_name(new_android_res_name)
testname = [[word, res_name[word]]for word in res_name]


sort_android_name=sorted(testname, key=lambda x: x[1], reverse=True)
sort_android_name
#套用上序方法做計數,並使用sorted做排序